In [1]:
import pandas as pd
import numpy as np

In [2]:
# grand_table = pd.read_csv('input/grand_table.csv')

In [3]:
values_to_fill = pd.read_csv('missing_value_for_agg.csv')

In [4]:
def fill_all_missing_values(grand_table):
    value_info = {}
    for index, row in values_to_fill.iterrows():
        table = row['Table']
        row_name = row['Row']
        row_type = row['Type']
        if not value_info.get(table):
            value_info[table] = {}
        value_info[table][row_name] = (row_type, row['missing value after aggregation'])
    
    for table in value_info.keys():
        # reverse sort so that "AB_CD" appears before "AB", requried for later processing
        columns = list(sorted(value_info[table].keys(), reverse=True))
        value_info[table]['all_columns'] = columns
        
    def fill_value(df, column, value):
        if value == 'median':
            df[column] = df[column].fillna(df[column].median())
        else:
            df[column] = df[column].fillna(int(value))
        return df
    
    def process_column(df, column, table):
        for col_name in value_info[table]['all_columns']:
            if col_name in column:
                return fill_value(df, column, value_info[table][col_name][1])
        print("Column %s in table %s not processed" % (column, table))
        
    def process(df):
        for column in df.columns:
            column = str(column)

            if not df[column].isnull().values.any():  # no missing value
                continue
            if column[0:len('prev_app')] == 'prev_app':
                df = process_column(df, column, 'previous_application.csv')
            elif column[0:len('install')] == 'install':
                df = process_column(df, column, 'installments_payments.csv')
            elif column[0:len('credit_bal')] == 'credit_bal':
                df = process_column(df, column, 'credit_card_balance.csv')
            elif column[0:len('pos_cash')] == 'pos_cash':
                if 'CNT_INSTALMENT_FUTURE' in column:  # this column is an ID, should drop
                    df = df.drop(column, axis=1)
                    continue
                df = process_column(df, column, 'POS_CASH_balance.csv')
            elif column[0:len('bureau')] == 'bureau':
                df = process_column(df, column, 'bureau.csv')
            elif column[0:len('client_bureau_balance')] == 'client_bureau_balance':
                # remove all columns from MONTHS_BALANCE except for 'MONTHS_BALANCE_min' because
                # MONTHS_BALANCE is like an ID, but its min has meaning
                if 'MONTHS_BALANCE' in column and 'MONTHS_BALANCE_min' not in column:
                    df = df.drop(column, axis=1)
                    continue
                df = process_column(df, column, 'bureau_balance.csv')
            else:
                print("column %s from grand table not processed" % column)
        return df
    
    return process(grand_table)

In [6]:
# grand_table = fill_all_missing_values(grand_table)

column TARGET from grand table not processed


In [4]:
# # adapted from a kernel:
# # https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
train = pd.read_csv("main_table.csv", nrows=400)
test = pd.read_csv("test_table.csv")
del train['TARGET']
del test['TARGET']
grand_table= train
grand_table_na = (grand_table.isnull().sum() / len(grand_table)) * 100
grand_table_na = grand_table_na.drop(grand_table_na[grand_table_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio': grand_table_na})
missing_data.head(20)

,Missing Ratio
